In [52]:
from langchain.chains.question_answering.map_rerank_prompt import output_parser

customer_review = """
We love this leaf blower. We use it on the porches of the house, and it's amazing. We also use it almost daily to blow up the goat dingleberries in our goat barn. It starts super easy, and lasts quite a while on one battery. It comes with two batteries, so we constantly have a battery ready when the other goes low. Absolutely love this blower, and would purchase it again. It arrives in two days.
"""

review_template = """
For the following text, extract the following information:
battery: was the item purchases with more than one battery?
deliver_days: How many days does it takes for the product to the costumer?
positive_rate: Does the customer gives a positive rate on this product?

Format the output as json with the following keys:
battery
deliver_days
positive_rate


text:{text}
"""

In [10]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(review_template)
prompt_template

ChatPromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nFor the following text, extract the following information:\nbattery: was the item purchases with more than one battery?\ndeliver_days: How many days does it takes for the product to the costumer?\npositive_rate: Does the customer gives a positive rate on this product?\n\ntext:{text}\n'), additional_kwargs={})])

In [11]:
from langchain_ollama import ChatOllama
chat = ChatOllama(model='llama3.2')

In [12]:
response = chat.invoke(prompt_template.format(text = customer_review))

In [13]:
response.content

'Here\'s the extracted information:\n\n- battery: Yes, the item was purchased with more than one battery (it came with two batteries).\n- deliver_days: 2 days\n- positive_rate: The customer gave a very positive rate, stating "Absolutely love this blower" and saying they would purchase it again.'

In [14]:
response.content.get('positive_rate')

AttributeError: 'str' object has no attribute 'get'

In [50]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
battery_schema = ResponseSchema(name='battery', description="was the item purchases with more than one battery? Output True if yes, Output False if not or unknown.")
deliver_days_schema = ResponseSchema(name='deliver_days', description="How many days does it takes for the product to the costumer? If unknown output -1")
positive_rate_schema = ResponseSchema(name='positive_rate', description="Does the customer gives a positive rate on this product? Output True if yes, Output False if not or unknown.")

response_schema = [battery_schema,deliver_days_schema, positive_rate_schema]

In [53]:
output_parser = StructuredOutputParser.from_response_schemas(response_schema)
format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"battery": string  // was the item purchases with more than one battery? Output True if yes, Output False if not or unknown.\n\t"deliver_days": string  // How many days does it takes for the product to the costumer? If unknown output -1\n\t"positive_rate": string  // Does the customer gives a positive rate on this product? Output True if yes, Output False if not or unknown.\n}\n```'

In [54]:
review_template2 = """
For the following text, extract the following information:

battery: was the item purchases with more than one battery?
deliver_days: How many days does it takes for the product to the costumer?
positive_rate: Does the customer gives a positive rate on this product?

text:{text}
{format_instructions}
"""

In [55]:
prompt_template2 = ChatPromptTemplate.from_template(review_template2)
message2 = prompt_template2.format_messages(text = customer_review, format_instructions = format_instructions)

In [56]:
message2

[HumanMessage(content='\nFor the following text, extract the following information:\n\nbattery: was the item purchases with more than one battery?\ndeliver_days: How many days does it takes for the product to the costumer?\npositive_rate: Does the customer gives a positive rate on this product?\n\ntext:\nWe love this leaf blower. We use it on the porches of the house, and it\'s amazing. We also use it almost daily to blow up the goat dingleberries in our goat barn. It starts super easy, and lasts quite a while on one battery. It comes with two batteries, so we constantly have a battery ready when the other goes low. Absolutely love this blower, and would purchase it again. It arrives in two days\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"battery": string  // was the item purchases with more than one battery? Output True if yes, Output False if not or unknown.\n\t"deliver_da

In [57]:
response2 = chat.invoke(message2)
response2

AIMessage(content='```json\n{\n    "battery": true,\n    "deliver_days": "-1",\n    "positive_rate": true\n}\n```\n\nExplanation of the extraced information:\n\n- The item was purchased with more than one battery, so `battery` is `true`.\n- We don\'t have any information on how many days it takes for the product to reach the customer. Therefore `deliver_days` is `-1`\n- The customer gives a positive rate on this product, so `positive_rate` is `true`.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-12-01T09:16:09.2826567Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3428292800, 'load_duration': 1637500200, 'prompt_eval_count': 294, 'prompt_eval_duration': 308000000, 'eval_count': 105, 'eval_duration': 1480000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-74442ff2-016c-40ee-be56-eefce76c38f0-0', usage_metadata={'input_tokens': 294, 'output_tokens': 105, 'total_tokens': 399})

In [47]:
response2.content

'Here is the extracted information:\n\n- battery: TRUE (the item comes with more than one battery)\n- deliver_days: 2 days\n- positive_rate: TRUE (the customer gave a positive review and would purchase it again)'

In [41]:
response2 = chat.invoke(prompt_template2.format_messages(text = customer_review, format_instructions = format_instructions))
response2.content

'Here is the extracted information:\n\n- battery: Yes, the product was purchased with more than one battery (comes with 2 batteries).\n- deliver_days: The product takes 2 days to arrive at the customer.\n- positive_rate: The customer gave a very positive rate, stating they "absolutely love" the product and would purchase it again.'

In [58]:
output_dict = output_parser.parse(response2.content)
output_dict

{'battery': True, 'deliver_days': '-1', 'positive_rate': True}

In [59]:
output_dict.get('battery')

True